In [1]:
import os
import numpy as np
import cv2
import torch
import torchvision
import matplotlib.pylab as plt

DATAS_UP_PATH='./output_up_camera/'
DATAS_DOWN_PATH='./output_down_camera/'
DATAS_STOP = './output_down_camera_stop_people/'
DATAS_NVIDIA = '/output_down_camera_stop_people/'

In [2]:
up_list = os.listdir(DATAS_UP_PATH)
donw_list = os.listdir(DATAS_DOWN_PATH)
stop_list=os.listdir(DATAS_STOP)

In [3]:
dataset=list()
train_list=donw_list
PATH = DATAS_DOWN_PATH
# 0.4~ 0.6 / ~0.4 / 0.6~
CLASSES_NUM = 4 # go, left, right, people
label_vector = np.eye(CLASSES_NUM)
for file_name in train_list:
    if file_name[-4:] != '.png':continue    
    img = cv2.imread(PATH+file_name)
    img = img.reshape(3,224,224)
    x = float(file_name.split('_')[1])
    if x >0.45 and x <0.55:
        label = torch.from_numpy(label_vector[0])
        dataset.append((img, label))

    elif x <= 0.45:
        label = torch.from_numpy(label_vector[1])
        dataset.append((img, label))
        label_c = torch.from_numpy(label_vector[2])
        img_c = img.copy()
        img_c = cv2.flip(img_c,-1)
        dataset.append((img_c, label_c))
    else:
        label = torch.from_numpy(label_vector[2])
        dataset.append((img, label))
        label_c = torch.from_numpy(label_vector[1])
        img_c = img.copy()
        img_c = cv2.flip(img_c,-1)
        dataset.append((img_c, label_c))
## 사람 etc
for file_name in stop_list:
    if file_name[-4:] != '.png':continue    
    img = cv2.imread(DATAS_STOP+file_name)
    img = img.reshape(3,224,224)
    dataset.append((img, torch.from_numpy(label_vector[3])))    

In [4]:
TEST_RATE = 0.2
BATCH_SIZE = 8
TEST_SET_NUM = int(len(dataset)*TEST_RATE)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - TEST_SET_NUM, TEST_SET_NUM])

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
#     num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
#     num_workers=4
)

In [6]:
class AutoDrive(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.output_classes=4 # position_bool, people, x, y
        self.model_up = self.making_transfer_model1()

    
    def making_transfer_model1(self):
        model = torchvision.models.squeezenet1_1(pretrained=True)       
        model.classifier=torch.nn.Sequential(
            torch.nn.Dropout(p=0.5, inplace=True),
            torch.nn.Conv2d(512, self.output_classes, kernel_size=(1,1), stride=(1,1)),
            torch.nn.ReLU(True),
            torch.nn.AdaptiveAvgPool2d(output_size=(1,1)),
            torch.nn.Flatten()
        )
        return model
    
#     def making_transfer_model2(self):
#         model = models.squeezenet1_0(pretrained=True)
#         model.classifier=torch.nn.Sequential(
# #             torch.nn.Dropout(p=0.5, inplace=True),
# #             torch.nn.Linear(in_features=212992, out_features=self.unit_output, bias=True),
# #             torch.nn.Mish(),
#             torch.nn.Dropout(p=0.5, inplace=True),
#             torch.nn.Conv2d(512, self.unit_output, kernel_size=(1,1), stride=(1,1)),
#             torch.nn.ReLU(True),
#             torch.nn.AdaptiveAvgPool2d(output_size=(1,1)),
#             torch.nn.Flatten()
#         )
#         return model

    def forward(self, up_img): # follow / to_position / line_num / loaded
        # up_img = torch.from_numpy(up_img).float()
        # down_img = torch.from_numpy(down_img).float()
        x_up = self.model_up(up_img)
        # x_down = self.model_down(down_img)
        # follow = follow.unsqueeze(-1)
        # to_position = to_position.unsqueeze(-1)
        # line_num = line_num.unsqueeze(-1)
        # loaded = loaded.unsqueeze(-1)
        # x = torch.cat([follow,to_position, line_num, loaded, x_up, x_down], dim=-1)
        # x = self.determine_model(x.float())
        return x_up


In [7]:
device = 'cuda'
auto = AutoDrive()
auto = auto.to(device=device)

In [8]:
import datetime
from torch.utils.tensorboard import SummaryWriter
LEARNING_RATE = 1e-4 # 1e-3
NUM_EPOCHS=100*1000
optimizer = torch.optim.Adam(auto.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9), weight_decay=0.0005)
# scaler = torch.cuda.amp.GradScaler()
BEST_MODEL_PATH = 'best_model_squeeze_down3.pth'

criterion = torch.nn.SmoothL1Loss()

d_time = datetime.datetime.now()
folder_name = "runs/"+d_time.strftime("%Y%m%d%H%M%S")
os.mkdir(folder_name)
writer = SummaryWriter(log_dir=folder_name)
pre_test_loss = 10.
EARLY_BIRD = 0
for epoch in range(NUM_EPOCHS):
    train_loss=0.
    test_loss=0.
#     auto.train()
    for up_img,labels in train_loader:
#         print(up_img)
#         up_img = normalize(up_img.float()).to(device=device)
        up_img = up_img.float().to(device=device)
        labels=labels.to(device);                
        optimizer.zero_grad()
        outputs = auto(up_img.float())
#         print("outputs: ",outputs[-1], "labels: ", labels[-1])
#         loss = F.cross_entropy(outputs, labels)
        loss = criterion(outputs, labels)
        train_loss+=loss        
        loss.backward()
        optimizer.step()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
        
    
    acc = 0.
    acc0=acc1=acc2=0
    count0=count1=count2=0
    with torch.no_grad():
        for up_img,labels in test_loader:
#             up_img = normalize(up_img.float()).to(device=device);
            up_img = up_img.float().to(device=device)
            labels=labels.to(device);        1        
            outputs = auto(up_img.float())
    #         loss_position = F.cross_entropy(outputs[:,0], position_bool)
    #         loss_people = F.cross_entropy(outputs[:,1], people)
    #         loss_x = F.cross_entropy(outputs[:,2], x)
    #         loss_y = F.cross_entropy(outputs[:,3], y)
            loss =criterion(outputs, labels)
            acc0 += sum(labels.argmax(-1)==0)
            acc1 += sum(labels.argmax(-1)==1)
            acc2 += sum(labels.argmax(-1)==2)
            count0 += sum(outputs.argmax(-1)==0)
            count1 += sum(outputs.argmax(-1)==1)
            count2 += sum(outputs.argmax(-1)==2)
                
            acc += sum(outputs.argmax(-1)==labels.argmax(-1))
            test_loss+=loss
            
#             print("outputs: ",outputs)
#     print(f"{epoch} TRAIN_LOSS: {train_loss.item()} TEST LOSS: {test_loss.item()}")
    writer.add_scalar("ACC",acc.item()/len(test_dataset),epoch)
    writer.add_scalar("TEST LOSS",test_loss.item()/len(test_dataset),epoch)
    # print(epoch, "TRAIN_LOSS",train_loss.item()/len(test_dataset),"TEST LOSS",test_loss.item()/len(test_dataset), 'ACC:', acc.item()/len(test_dataset))
    # print(acc0.item(),count0.item(),"  ", acc1.item(), count1.item(), "  ", acc2.item(), count2.item())
   
#     print("outputs: ",outputs[-1], "labels: ", labels[-1])
    
    if pre_test_loss>test_loss and epoch>50:
        pre_test_loss= test_loss
        torch.save(auto.state_dict(), BEST_MODEL_PATH)
    if pre_test_loss<test_loss and epoch>50:
        EARLY_BIRD +=1
        if EARLY_BIRD >100:
            break